# LLM Skyline
GPT has demonstrated excellent performance on the task using in-context learning. Here, we run a systematic evaluation, in order to provide a *skyline*, i.e. near-optimal automated system. Of course, making OpenAI API calls might not be ideal in real usage for a number of reasons.

In [1]:
import pandas as pd
import os

# Read all of the data into a single combined DF
folder_path = '../americasnlp2024/ST2_EducationalMaterials/data/'
all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith('.tsv'):
        split_df = pd.read_csv(os.path.join(folder_path, filename), delimiter='\t')
        [split_df['language'], split_df['split']] = filename[:-4].split("-")
        all_data.append(split_df)

df = pd.concat(all_data, ignore_index=True)

# We'll add spaces between letters to avoid tokenization issues
df['Formatted'] = df.apply(lambda row: f"Id: {row['ID']}\nSource: {row['Source']}\nChange: {row['Change']}\nTarget: {row['Target']}", axis=1)
df['Formatted_Covered'] = df.apply(lambda row: f"Id: {row['ID']}\nSource: {row['Source']}\nChange: {row['Change']}\nTarget: ", axis=1)
df['Predicted Target'] = ''
df

,ID,Source,Change,Target,language,split,Formatted,Formatted_Covered,Predicted Target
0,Maya0119,Tene' áak'ab kin bin merkaado,TYPE:NEG,Tene' ma' áak'ab kin bin merkaadoi',maya,train,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,
1,Maya0120,Tene' áak'ab kin bin merkaado,SUBTYPE:INT,Tene' wáaj áak'ab kin bin merkaado,maya,train,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,
2,Maya0317,Ko'one'ex ich kool,PERSON:1_PL,Ko'ox ich kool,maya,train,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,
3,Maya0620,Táan a bine'ex ich kool,TYPE:NEG,Ma' táan a bine'ex ich kooli',maya,train,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,
4,Maya0621,Táan a bine'ex ich kool,SUBTYPE:INT,Táan wáaj a bine'ex ich kool,maya,train,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,
...,...,...,...,...,...,...,...,...,...
1516,Bribri0676,Ye' tö dawà su',"TYPE:NEG, TENSE:FUT_CER, ASPECT:IPFV, VOICE:MI...",Kë̀ dawà sùrpa,bribri,dev,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,
1517,Bribri0677,Ye' tö dawà su',TENSE:PAS_PLU,Ye' wa̠ dawà súrule,bribri,dev,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,
1518,Bribri0678,Ye' tö dawà su',"TYPE:NEG, TENSE:PAS_PLU",Ye' kë̀ wa̠ dawà súrule,bribri,dev,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,
1519,Bribri0679,Ye' tö dawà su',"MODE:POT, TENSE:IPFV_REC, ASPECT:IPFV",Ye' a̠ dawà súr,bribri,dev,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,


In [84]:
import re
def remove_single_spaces(text: str) -> str:
    # Replace single spaces between letters with no space
    return re.sub("\s+", " ", re.sub(r'(?<=\w|\') (?=\w|\')', '', text))

# Fixes unattached diacritics
def attach_diacritics(text: str) -> str:
    # Function to reorder each match
    def reorder(match):
        char, diacritic = match.groups()
        # Return the reordered string with the diacritic attached to the character
        return char + diacritic
    
    # Regular expression to find a character followed by a space and then the diacritic
    pattern = r'(\w) ([ ̀ ̠])'
    # Replace occurrences found by the pattern with the reordered version
    adjusted_text = re.sub(pattern, reorder, text)
    
    return adjusted_text

remove_single_spaces(attach_diacritics("P û s   k ë ̀   k u ̠   k a p ë ' w a ̠"))

"Pûs kë̀ ku̠ kapë'wa̠"

## Naive ICL

In [2]:
from openai import OpenAI
import re
from tqdm.notebook import tqdm

if 'api_key' not in vars():
    api_key = input("OpenAI API Key:")

client = OpenAI(api_key=api_key)


def run_prompt_full_context(lang, log_file, test_IDs=None):
    """Runs a GPT prompt for a specified row in the dev/test set. Uses the entire `train` split as context.

    Args:
        lang: 'bribri' | 'guarani' | 'maya'
        test_ID The ID of a row in the dev/test set to run inference on.
    """
    train_split = df[(df['language'] == lang) & (df['split'] == 'train')]

    if test_IDs is not None:
        test_sentences = df[df['ID'].isin(test_IDs)]
    else:
        test_sentences = df[(df['language'] == lang) & (df['split'] == 'dev')]

    system_prompt = f"You are an expert in the {lang.capitalize()} language. You are creating education materials by taking a given sentence in {lang.capitalize()} and a label indicating a change in one or more linguistic features, and outputting the sentence transformed by changing that feature."
    context = '\n\n'.join(train_split['Formatted'])
    test_examples = '\n\n'.join(test_sentences['Formatted_Covered'])


    prompt = f"""Below are examples of a sentence in {lang.capitalize()}, the linguistic change, and the target sentence after applying the change.
    
{context}

Below is a list of similar examples, where the source sentence and linguistic change are given, and the output sentence is not known. For each example, please output only the id and target sentence values, as in

ID: some id
Target: sentence after applying the change


Do not output any additional text, and do not output the Source or Change fields. This is very important, take your time and do not mess up or I will lose my job.

{test_examples}
    """

    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        top_p=1,
        seed=430
    )
    print(completion.usage)
    print(completion.model)

    pattern = r"I[dD]: (\S+)\nTarget: (.*)(\n|$)"
    resp = completion.choices[0].message.content
    
    with open(log_file, 'a') as log:
        log.write("\n\nPROMPT:\n" + prompt)
        log.write("\nRESPONSE:\n" + resp)

    matches = re.findall(pattern, resp, re.M)
    matches_dict = dict()
    for match in matches:
        matches_dict[match[0]] = match[1] # remove_single_spaces(attach_diacritics(match[1]))
    return matches_dict


In [3]:
import math

def test_full_context(chunk_size, df):
    df = df.copy(deep=True)
    for language in tqdm(['bribri', 'guarani', 'maya']):
        lang_test_size = len(df[(df['language'] == language) & (df['split'] == 'dev')])

        for chunk in tqdm(range(math.ceil(lang_test_size / chunk_size))):
            print(f"Testing indices {chunk*chunk_size} through {(chunk+1)*chunk_size}")
            test_chunk = df[(df['language'] == language) & (df['split'] == 'dev')]['ID'].values[chunk*chunk_size: (chunk+1)*chunk_size]
            pred_dict = run_prompt_full_context(lang=language, test_IDs=test_chunk, log_file=f"./{language}.log")
            for pred_id, pred_string in pred_dict.items():
                df.loc[df['ID'] == pred_id, 'Predicted Target'] = pred_string

    df[(df['language'] == 'bribri') & (df['split'] == 'dev')].to_csv(f"../preds/chatgpt/fc_chunksize_{chunk_size}/bribri-dev-preds.tsv", sep="\t")
    df[(df['language'] == 'guarani') & (df['split'] == 'dev')].to_csv(f"../preds/chatgpt/fc_chunksize_{chunk_size}/guarani-dev-preds.tsv", sep="\t")
    df[(df['language'] == 'maya') & (df['split'] == 'dev')].to_csv(f"../preds/chatgpt/fc_chunksize_{chunk_size}/maya-dev-preds.tsv", sep="\t")

    return df

preds = test_full_context(chunk_size=80, df=df)

preds

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Testing indices 0 through 80
CompletionUsage(completion_tokens=1814, prompt_tokens=19955, total_tokens=21769)
gpt-4-0125-preview
Testing indices 80 through 160
CompletionUsage(completion_tokens=1768, prompt_tokens=20076, total_tokens=21844)
gpt-4-0125-preview
Testing indices 160 through 240
CompletionUsage(completion_tokens=1177, prompt_tokens=18937, total_tokens=20114)
gpt-4-0125-preview


  0%|          | 0/1 [00:00<?, ?it/s]

Testing indices 0 through 80
CompletionUsage(completion_tokens=1709, prompt_tokens=10973, total_tokens=12682)
gpt-4-0125-preview


  0%|          | 0/2 [00:00<?, ?it/s]

Testing indices 0 through 80
CompletionUsage(completion_tokens=4096, prompt_tokens=31171, total_tokens=35267)
gpt-4-0125-preview
Testing indices 80 through 160
CompletionUsage(completion_tokens=4096, prompt_tokens=30935, total_tokens=35031)
gpt-4-0125-preview


,ID,Source,Change,Target,language,split,Formatted,Formatted_Covered,Predicted Target
0,Maya0119,Tene' áak'ab kin bin merkaado,TYPE:NEG,Tene' ma' áak'ab kin bin merkaadoi',maya,train,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,
1,Maya0120,Tene' áak'ab kin bin merkaado,SUBTYPE:INT,Tene' wáaj áak'ab kin bin merkaado,maya,train,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,
2,Maya0317,Ko'one'ex ich kool,PERSON:1_PL,Ko'ox ich kool,maya,train,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,
3,Maya0620,Táan a bine'ex ich kool,TYPE:NEG,Ma' táan a bine'ex ich kooli',maya,train,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,
4,Maya0621,Táan a bine'ex ich kool,SUBTYPE:INT,Táan wáaj a bine'ex ich kool,maya,train,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,
...,...,...,...,...,...,...,...,...,...
1516,Bribri0676,Ye' tö dawà su',"TYPE:NEG, TENSE:FUT_CER, ASPECT:IPFV, VOICE:MI...",Kë̀ dawà sùrpa,bribri,dev,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,Kë̀ dawà sùrpa
1517,Bribri0677,Ye' tö dawà su',TENSE:PAS_PLU,Ye' wa̠ dawà súrule,bribri,dev,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,Ye' wa̠ dawà súrule
1518,Bribri0678,Ye' tö dawà su',"TYPE:NEG, TENSE:PAS_PLU",Ye' kë̀ wa̠ dawà súrule,bribri,dev,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,Ye' kë̀ wa̠ dawà súrule
1519,Bribri0679,Ye' tö dawà su',"MODE:POT, TENSE:IPFV_REC, ASPECT:IPFV",Ye' a̠ dawà súr,bribri,dev,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,Ye' a̠ dawà súr


In [4]:
preds = test_full_context(chunk_size=20, df=df)
preds

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Testing indices 0 through 20
CompletionUsage(completion_tokens=440, prompt_tokens=17559, total_tokens=17999)
gpt-4-0125-preview
Testing indices 20 through 40
CompletionUsage(completion_tokens=570, prompt_tokens=17673, total_tokens=18243)
gpt-4-0125-preview
Testing indices 40 through 60
CompletionUsage(completion_tokens=362, prompt_tokens=17589, total_tokens=17951)
gpt-4-0125-preview
Testing indices 60 through 80
CompletionUsage(completion_tokens=388, prompt_tokens=17567, total_tokens=17955)
gpt-4-0125-preview
Testing indices 80 through 100
CompletionUsage(completion_tokens=412, prompt_tokens=17576, total_tokens=17988)
gpt-4-0125-preview
Testing indices 100 through 120
CompletionUsage(completion_tokens=473, prompt_tokens=17615, total_tokens=18088)
gpt-4-0125-preview
Testing indices 120 through 140
CompletionUsage(completion_tokens=471, prompt_tokens=17599, total_tokens=18070)
gpt-4-0125-preview
Testing indices 140 through 160
CompletionUsage(completion_tokens=471, prompt_tokens=17719, t

  0%|          | 0/4 [00:00<?, ?it/s]

Testing indices 0 through 20
CompletionUsage(completion_tokens=375, prompt_tokens=9066, total_tokens=9441)
gpt-4-0125-preview
Testing indices 20 through 40
CompletionUsage(completion_tokens=496, prompt_tokens=9186, total_tokens=9682)
gpt-4-0125-preview
Testing indices 40 through 60
CompletionUsage(completion_tokens=412, prompt_tokens=9075, total_tokens=9487)
gpt-4-0125-preview
Testing indices 60 through 80
CompletionUsage(completion_tokens=386, prompt_tokens=9059, total_tokens=9445)
gpt-4-0125-preview


  0%|          | 0/8 [00:00<?, ?it/s]

Testing indices 0 through 20
CompletionUsage(completion_tokens=467, prompt_tokens=29256, total_tokens=29723)
gpt-4-0125-preview
Testing indices 20 through 40
CompletionUsage(completion_tokens=443, prompt_tokens=29227, total_tokens=29670)
gpt-4-0125-preview
Testing indices 40 through 60
CompletionUsage(completion_tokens=496, prompt_tokens=29279, total_tokens=29775)
gpt-4-0125-preview
Testing indices 60 through 80
CompletionUsage(completion_tokens=444, prompt_tokens=29248, total_tokens=29692)
gpt-4-0125-preview
Testing indices 80 through 100
CompletionUsage(completion_tokens=508, prompt_tokens=29299, total_tokens=29807)
gpt-4-0125-preview
Testing indices 100 through 120
CompletionUsage(completion_tokens=490, prompt_tokens=29276, total_tokens=29766)
gpt-4-0125-preview
Testing indices 120 through 140
CompletionUsage(completion_tokens=467, prompt_tokens=29244, total_tokens=29711)
gpt-4-0125-preview
Testing indices 140 through 160
CompletionUsage(completion_tokens=210, prompt_tokens=28955, t

,ID,Source,Change,Target,language,split,Formatted,Formatted_Covered,Predicted Target
0,Maya0119,Tene' áak'ab kin bin merkaado,TYPE:NEG,Tene' ma' áak'ab kin bin merkaadoi',maya,train,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,Id: Maya0119\nSource: Tene' áak'ab kin bin mer...,
1,Maya0120,Tene' áak'ab kin bin merkaado,SUBTYPE:INT,Tene' wáaj áak'ab kin bin merkaado,maya,train,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,Id: Maya0120\nSource: Tene' áak'ab kin bin mer...,
2,Maya0317,Ko'one'ex ich kool,PERSON:1_PL,Ko'ox ich kool,maya,train,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,Id: Maya0317\nSource: Ko'one'ex ich kool\nChan...,
3,Maya0620,Táan a bine'ex ich kool,TYPE:NEG,Ma' táan a bine'ex ich kooli',maya,train,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,Id: Maya0620\nSource: Táan a bine'ex ich kool\...,
4,Maya0621,Táan a bine'ex ich kool,SUBTYPE:INT,Táan wáaj a bine'ex ich kool,maya,train,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,Id: Maya0621\nSource: Táan a bine'ex ich kool\...,
...,...,...,...,...,...,...,...,...,...
1516,Bribri0676,Ye' tö dawà su',"TYPE:NEG, TENSE:FUT_CER, ASPECT:IPFV, VOICE:MI...",Kë̀ dawà sùrpa,bribri,dev,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0676\nSource: Ye' tö dawà su'\nChang...,Kë̀ dawà su̠rpa
1517,Bribri0677,Ye' tö dawà su',TENSE:PAS_PLU,Ye' wa̠ dawà súrule,bribri,dev,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0677\nSource: Ye' tö dawà su'\nChang...,Ye' wa̠ dawà su̠rule
1518,Bribri0678,Ye' tö dawà su',"TYPE:NEG, TENSE:PAS_PLU",Ye' kë̀ wa̠ dawà súrule,bribri,dev,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0678\nSource: Ye' tö dawà su'\nChang...,Ye' kë̀ wa̠ dawà su̠rule
1519,Bribri0679,Ye' tö dawà su',"MODE:POT, TENSE:IPFV_REC, ASPECT:IPFV",Ye' a̠ dawà súr,bribri,dev,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,Id: Bribri0679\nSource: Ye' tö dawà su'\nChang...,Ye' a̠ dawà su̠r


## Retrieval

Rather than passing full context, let's try selecting informative examples for each sentence. First, we'll split up sentences based on the linguistic change tags. Then, for each group of sentences, we'll retrieve train sentences with similar tags.

In [10]:
def run_prompt_naive_retrieval(lang, test_IDs):
    """Runs a GPT prompt for a specified row in the dev/test set. Retrieves items from the `train` split that have the same change tags.

    Args:
        lang: 'bribri' | 'guarani' | 'maya'
        test_ID The ID of a row in the dev/test set to run inference on.
    """
    train_split = df[(df['language'] == lang) & (df['split'] == 'train')]

    assert(test_IDs is not None)

    test_sentences = df[df['ID'].isin(test_IDs)]

    # Determine the tags that appear in the test sentences
    test_change_tags = test_sentences['Change'].unique()
    print(test_change_tags)
    filtered_train = train_split[train_split['Change'].isin(test_change_tags)]

    system_prompt = f"You are an expert in the {lang.capitalize()} language. You are creating education materials by taking a given sentence in {lang.capitalize()} and a label indicating a change in one or more linguistic features, and outputting the sentence transformed by changing that feature. All Bribri text is seperated by spaces."
    context = '\n\n'.join(filtered_train['Formatted'])
    test_examples = '\n\n'.join(test_sentences['Formatted_Covered'])


    prompt = f"""Below are examples of a sentence in {lang.capitalize()}, the linguistic change, and the target sentence after applying the change.
    
{context}

Below is a list of similar examples, where the source sentence and linguistic change are given, and the output sentence is not known. For each example, please output only the id and target sentence values, as in

ID: some id
Target: sentence after applying the change


Do not output any additional text, and do not output the Source or Change fields. This is very important, take your time and do not mess up or I will lose my job.

{test_examples}
    """

    print(prompt)

    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        top_p=1,
        seed=430
    )
    print(completion.usage)
    print(completion.model)

    pattern = r"I[dD]: (\S+)\nTarget: (.*)(\n|$)"
    resp = completion.choices[0].message.content
    print(resp)
    matches = re.findall(pattern, resp, re.M)
    matches_dict = dict()
    for match in matches:
        matches_dict[match[0]] = match[1]
    return matches_dict

run_prompt_naive_retrieval("bribri", test_IDs=["Bribri0362", "Bribri0367"])


['TENSE:PRF_REC, ABSNUM:PL' 'TYPE:NEG, TENSE:PRF_PROG']
Below are examples of a sentence in Bribri, the linguistic change, and the target sentence after applying the change.
    
Id: Bribri0303
Source: Y e '   s h k a '
Change: TYPE:NEG, TENSE:PRF_PROG
Target: Y e '   k ë ̀   k u ̠ ' b a k   s h k ö ́ k

Id: Bribri0962
Source: Y e '   t ö   i   s í k
Change: TYPE:NEG, TENSE:PRF_PROG
Target: Y e '   k ë ̀   k u ̠ ' b a k   i   s k ö ́ k

Id: Bribri0771
Source: Y e '   d ö r   à r r o s   t ó ̠   ñ è ̠
Change: TYPE:NEG, TENSE:PRF_PROG
Target: Y e '   k ë ̀   k u ̠ ' b a k   à r r o s   t a ̠ ú ̠ k   ñ è ̠

Id: Bribri0493
Source: Y e '   t ö   k à s i r   t ë '
Change: TYPE:NEG, TENSE:PRF_PROG
Target: Y e '   k ë ̀   k u ̠ ' b a k   k à s i r   t ö ́ k

Id: Bribri0899
Source: I e '   t ö   b ö '   y ë ' s t s a ̠
Change: TENSE:PRF_REC, ABSNUM:PL
Target: I e '   t ö   b ö '   y é u l u r

Id: Bribri0380
Source: I e '   d ú w a ̠
Change: TYPE:NEG, TENSE:PRF_PROG
Target: I e '   k ë ̀   k u 

NameError: name 'client' is not defined

In [48]:
def run_prompt_smart_retrieval(lang, log_file, test_IDs):
    """Runs a GPT prompt for a specified row in the dev/test set. Retrieves items from the `train` split that have the same change tags and sub-change tags

    Args:
        lang: 'bribri' | 'guarani' | 'maya'
        test_ID The ID of a row in the dev/test set to run inference on.
    """
    train_split = df[(df['language'] == lang) & (df['split'] == 'train')]

    assert(test_IDs is not None)

    test_sentences = df[df['ID'].isin(test_IDs)]

    for test_sent in test_sentences.iterrows():
        test_sent=test_sent[1]

        # Determine the tags that appear in the test sentences
        test_change_tags = test_sent['Change'].split(", ")
        test_change_tags.append(test_sent['Change'])
        print(test_change_tags)
        filtered_train = train_split[train_split['Change'].isin(test_change_tags)]

        system_prompt = f"You are an expert in the {lang.capitalize()} language. You are creating education materials by taking a given sentence in {lang.capitalize()} and a label indicating a change in one or more linguistic features, and outputting the sentence transformed by changing that feature. All Bribri text is seperated by spaces."
        context = '\n\n'.join(filtered_train['Formatted'])
        test_examples = '\n\n' + test_sent['Formatted_Covered']


        prompt = f"""Below are examples of a sentence in {lang.capitalize()}, the linguistic change, and the target sentence after applying the change.
        
    {context}

    Below is a similar example, where the source sentence and linguistic change are given, and the output sentence is not known. For this example, please output only the id and target sentence values, as in

    ID: some id
    Target: sentence after applying the change


    Do not output any additional text, and do not output the Source or Change fields. This is very important, take your time and do not mess up or I will lose my job.

    {test_examples}
    """

    completion = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        top_p=1,
        seed=430
    )
    print(completion.usage)
    print(completion.model)

    pattern = r"I[dD]: (\S+)\nTarget: (.*)(\n|$)"
    resp = completion.choices[0].message.content
    
    with open(log_file, 'a') as log:
        log.write("\n\nPROMPT:\n" + prompt)
        log.write("\nRESPONSE:\n" + resp)

    matches = re.findall(pattern, resp, re.M)
    matches_dict = dict()
    for match in matches:
        matches_dict[match[0]] = remove_single_spaces(attach_diacritics(match[1]))
    return matches_dict

run_prompt_smart_retrieval("bribri", "test.log", test_IDs=["Bribri0362", "Bribri0367"])

['TENSE:PRF_REC', 'ABSNUM:PL', 'TENSE:PRF_REC, ABSNUM:PL']
['TYPE:NEG', 'TENSE:PRF_PROG', 'TYPE:NEG, TENSE:PRF_PROG']


NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4-turbo-preview` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [ ]:
import math

def test_smart_context(chunk_size, df):
    df = df.copy(deep=True)
    for language in tqdm(['bribri', 'guarani', 'maya']):
        lang_test_size = len(df[(df['language'] == language) & (df['split'] == 'dev')])

        for chunk in tqdm(range(math.ceil(lang_test_size / chunk_size))):
            print(f"Testing indices {chunk*chunk_size} through {(chunk+1)*chunk_size}")
            test_chunk = df[(df['language'] == language) & (df['split'] == 'dev')]['ID'].values[chunk*chunk_size: (chunk+1)*chunk_size]
            pred_dict = run_prompt_smart_retrieval(lang=language, test_IDs=test_chunk, log_file=f"./{language}-smart.log")
            for pred_id, pred_string in pred_dict.items():
                df.loc[df['ID'] == pred_id, 'Predicted Target'] = pred_string

    df[(df['language'] == 'bribri') & (df['split'] == 'dev')].to_csv("bribri-dev-preds-smart.tsv", sep="\t")
    df[(df['language'] == 'guarani') & (df['split'] == 'dev')].to_csv("guarani-dev-preds-smart.tsv", sep="\t")
    df[(df['language'] == 'maya') & (df['split'] == 'dev')].to_csv("maya-dev-preds-smart.tsv", sep="\t")

    return df

preds = test_smart_context(chunk_size=80, df=df)

preds